# Text Generation Using Recurrent Neural Networks

This week we are going to learn about Recurrent Neural Networks (RNNs). RNNs are very popular in the field of natural language processing, because they are remarkably good at modelling langauge. This is because, unlike the other models we have looked at so far, RNNs take a account of the *order* in which things appear, and word order is obviously a very important feature of language. Because of their ability to take account of the *order* of observations, RNNs are a type of 'sequence model.'

I explained how these models work in the workshop. The [slides](slides/text-generation.pdf) are available in this repo.

To see how RNNs work, and to learn a bit abou what they are capable of, today we will train the computer to generate poetry based on 3 million lines of poetry from Project Gutenberg.

**NB:** If you are using this notebook in Google Colab, pleaes make sure to click the 'runtime' button in the top right, and choose 'GPU'.

## Import the poetry data

In [ ]:
# Download the 'Gutenberg Poetry Corpus' from Allison Parrish's website:
!curl -O http://static.decontextualize.com/gutenberg-poetry-v001.ndjson.gz

# Extract all the lines of poetry from the corpus
import gzip, json
all_lines = []
for line in gzip.open("gutenberg-poetry-v001.ndjson.gz"):
    all_lines.append(json.loads(line.strip()))

# Take a look
import random
random.sample(all_lines, 5)

# Preprocess the Data

## Define and train the network


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, 

# Generate Some Poetry